<a href="https://colab.research.google.com/github/Ansalnakhan/Medical-Image-Assgnmt/blob/main/project%20begining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
!mkdir dataset
!cd dataset
!unzip /content/drive/MyDrive/naza.zip -d /content/dataset

mkdir: cannot create directory ‘dataset’: File exists
Archive:  /content/drive/MyDrive/naza.zip
replace /content/dataset/B0007.mat? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [49]:
import datetime
import numpy as np
import pandas as pd

In [51]:
from scipy.io import loadmat
model.add(Flatten())

In [52]:
from sklearn.metrics import mean_squared_error
from sklearn import metrics
import matplotlib.pyplot as plt

In [53]:
from sklearn.metrics import confusion_matrix, accuracy_score, mean_absolute_error,r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential

In [54]:
from tensorflow.keras.layers import Dense,Conv1D

In [55]:
from tensorflow.keras.layers import Dropout,BatchNormalization,Bidirectional,MaxPooling1D

In [56]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model,Input

In [57]:
def load_data(battery):
  mat = loadmat('./dataset/' + battery + '.mat')
  print('Total data in dataset: ', len(mat[battery][0, 0]['cycle'][0]))
  counter = 0
  dataset = []
  capacity_data = []
  
  for i in range(len(mat[battery][0, 0]['cycle'][0])):
    row = mat[battery][0, 0]['cycle'][0, i]
    if row['type'][0] == 'discharge':
      ambient_temperature = row['ambient_temperature'][0][0]
      date_time = datetime.datetime(int(row['time'][0][0]),
                               int(row['time'][0][1]),
                               int(row['time'][0][2]),
                               int(row['time'][0][3]),
                               int(row['time'][0][4])) + datetime.timedelta(seconds=int(row['time'][0][5]))
      data = row['data']
      capacity = data[0][0]['Capacity'][0][0]
      for j in range(len(data[0][0]['Voltage_measured'][0])):
        voltage_measured = data[0][0]['Voltage_measured'][0][j]
        current_measured = data[0][0]['Current_measured'][0][j]
        temperature_measured = data[0][0]['Temperature_measured'][0][j]
        current_load = data[0][0]['Current_load'][0][j]
        voltage_load = data[0][0]['Voltage_load'][0][j]
        time = data[0][0]['Time'][0][j]
        dataset.append([counter + 1, ambient_temperature, date_time, capacity,
                        voltage_measured, current_measured,
                        temperature_measured, current_load,
                        voltage_load, time])
      capacity_data.append([counter + 1, ambient_temperature, date_time, capacity])
      counter = counter + 1
  print(dataset[0])
  return [pd.DataFrame(data=dataset,
                       columns=['cycle', 'ambient_temperature', 'datetime',
                                'capacity', 'voltage_measured',
                                'current_measured', 'temperature_measured',
                                'current_load', 'voltage_load', 'time']),
          pd.DataFrame(data=capacity_data,
                       columns=['cycle', 'ambient_temperature', 'datetime',
                                'capacity'])]

In [58]:
dataset_val, capacity_val = load_data('B0005')
attrib=['capacity', 'cycle']
dis_ele = capacity_val[attrib]
rows=['capacity','cycle']
dataset=dis_ele[rows]
data_train=dataset[(dataset['cycle']<90)]
data_set_train=data_train.iloc[:,1:2].values
data_test=dataset[(dataset['cycle']<=10)]
data_set_test=data_test.iloc[:,1:2].values

from sklearn.preprocessing import MinMaxScaler
sc=MinMaxScaler(feature_range=(0,1))
data_set_train=sc.fit_transform(data_set_train)
data_set_test=sc.transform(data_set_test)

X_train=[]
y_train=[]
#created window of 10
for i in range(10,89):
    X_train.append(data_set_train[i-10:i,0])
    y_train.append(data_set_train[i,0])
X_train,y_train=np.array(X_train),np.array(y_train)
 
X_train=np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))

Total data in dataset:  616
[1, 24, datetime.datetime(2008, 4, 2, 15, 25, 41), 1.8564874208181574, 4.191491807505295, -0.004901589207462691, 24.330033885570543, -0.0006, 0.0, 0.0]


In [59]:
dataset_val, capacity_val

(       cycle  ambient_temperature            datetime  capacity  \
 0          1                   24 2008-04-02 15:25:41  1.856487   
 1          1                   24 2008-04-02 15:25:41  1.856487   
 2          1                   24 2008-04-02 15:25:41  1.856487   
 3          1                   24 2008-04-02 15:25:41  1.856487   
 4          1                   24 2008-04-02 15:25:41  1.856487   
 ...      ...                  ...                 ...       ...   
 50280    168                   24 2008-05-27 20:45:42  1.325079   
 50281    168                   24 2008-05-27 20:45:42  1.325079   
 50282    168                   24 2008-05-27 20:45:42  1.325079   
 50283    168                   24 2008-05-27 20:45:42  1.325079   
 50284    168                   24 2008-05-27 20:45:42  1.325079   
 
        voltage_measured  current_measured  temperature_measured  current_load  \
 0              4.191492         -0.004902             24.330034       -0.0006   
 1              4.

In [96]:
model = Sequential()
model.add(Conv1D(filters=256, kernel_size=3, input_shape=(10, 1)))
model.add(MaxPooling1D(pool_size=2 ))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(16, input_dim=10, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='relu'))
model.add(Dense(1, activation='softmax'))  

In [78]:
model = Sequential()
model.add(Conv1D(filters=256, kernel_size=3, padding='same', activation='relu'))
model.add(Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(128, input_dim=10, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='softmax'))


In [34]:
model=Sequential()

In [39]:
model.add(Conv1D(filters=256, kernel_size=3, padding='same', activation='relu'))

In [40]:
model.add(MaxPooling1D(pool_size=2))

In [42]:
from tensorflow.keras.layers import Flatten

In [43]:
model.add(Flatten())
model.add(Dense(units=1))

In [97]:
adam = tf.keras.optimizers.Adam(
    learning_rate=8e-4)

In [98]:
model.compile(optimizer=adam,loss=tf.keras.losses.Huber(),metrics=tf.keras.metrics.RootMeanSquaredError())


In [99]:
model.fit(X_train, y_train, epochs=50, batch_size=128)

Epoch 1/50
1/1 [==============================] - 1s 828ms/step - loss: 0.1318 - root_mean_squared_error: 0.5134
Epoch 2/50
1/1 [==============================] - 0s 15ms/step - loss: 0.1318 - root_mean_squared_error: 0.5134
Epoch 3/50
1/1 [==============================] - 0s 17ms/step - loss: 0.1318 - root_mean_squared_error: 0.5134
Epoch 4/50
1/1 [==============================] - 0s 14ms/step - loss: 0.1318 - root_mean_squared_error: 0.5134
Epoch 5/50
1/1 [==============================] - 0s 15ms/step - loss: 0.1318 - root_mean_squared_error: 0.5134
Epoch 6/50
1/1 [==============================] - 0s 16ms/step - loss: 0.1318 - root_mean_squared_error: 0.5134
Epoch 7/50
1/1 [==============================] - 0s 16ms/step - loss: 0.1318 - root_mean_squared_error: 0.5134
Epoch 8/50
1/1 [==============================] - 0s 16ms/step - loss: 0.1318 - root_mean_squared_error: 0.5134
Epoch 9/50
1/1 [==============================] - 0s 17ms/step - loss: 0.1318 - root_mean_squared_error

In [76]:
from keras.models import Sequential

In [77]:
predictions = model.predict_proba(X_test)

AttributeError: ignored